In [ ]:
import dask.array as da
import numpy as np

import healpix_convolution.neighbours as nb

In [ ]:
from distributed import Client

client = Client()
client

In [ ]:
resolution = 6
cell_ids = da.arange(12 * 4**resolution, chunks=(100000,), dtype="int32")
cell_ids

In [ ]:
def dask_neighbours(cell_ids, *, resolution, indexing_scheme, ring):
    n_neighbours = (2 * ring + 1) ** 2
    return da.map_blocks(
        nb.neighbours,
        cell_ids,
        resolution=resolution,
        indexing_scheme=indexing_scheme,
        ring=ring,
        new_axis=1,
        chunks=cell_ids.chunks + (n_neighbours,),
        dtype=cell_ids.dtype,
    )

In [ ]:
%%time
arr = dask_neighbours(cell_ids, resolution=resolution, indexing_scheme="nested", ring=4)
arr

In [ ]:
%%time
arr.compute()

In [ ]:
%%time
nb.neighbours(
    cell_ids.compute(), resolution=resolution, indexing_scheme="nested", ring=4
)